In [1]:
!pip install boto3

# We are using policy universe from Netflix Skunkworks
!pip install policyuniverse

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.1/139.1 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.6/12.6 MB 57.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.6/82.6 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 484.3/484.3 kB 5.8 MB/s eta 0:00:00


Look at this policy in particular it has allowed all IAM actions beginning with create

In [4]:
policy_1 = {
        "Statement": [{
            "Action": ["iam:Create*"],
            "Resource": "*",
            "Effect": "Allow"
          }]
      }

# A static tool would have a challenging time parsing the impact of that without a sophisticated pattern
# We can expand it though to show the full impact

from policyuniverse.expander_minimizer import expand_policy
from pprint import pprint

expanded_policy = expand_policy(policy=policy_1)
print(pprint(expanded_policy))

{'Statement': [{'Action': ['iam:createaccesskey',
                           'iam:createaccountalias',
                           'iam:creategroup',
                           'iam:createinstanceprofile',
                           'iam:createloginprofile',
                           'iam:createopenidconnectprovider',
                           'iam:createpolicy',
                           'iam:createpolicyversion',
                           'iam:createrole',
                           'iam:createsamlprovider',
                           'iam:createservicelinkedrole',
                           'iam:createservicespecificcredential',
                           'iam:createuser',
                           'iam:createvirtualmfadevice'],
                'Effect': 'Allow',
                'Resource': '*'}]}
None


That really better and much more descriptive than the original.  We can however go back to the original for easy enough.

In [5]:
from policyuniverse.expander_minimizer import minimize_policy

minimized_policy = minimize_policy(policy=expanded_policy, minchars=3)
print(pprint(minimized_policy))

{'Statement': [{'Action': ['iam:cre*'], 'Effect': 'Allow', 'Resource': '*'}]}
None


Skipping prefix c because length of 1
Skipping prefix cr because length of 2
Skipping prefix c because length of 1
Skipping prefix cr because length of 2
Skipping prefix c because length of 1
Skipping prefix cr because length of 2
Skipping prefix c because length of 1
Skipping prefix cr because length of 2
Skipping prefix c because length of 1
Skipping prefix cr because length of 2
Skipping prefix c because length of 1
Skipping prefix cr because length of 2
Skipping prefix c because length of 1
Skipping prefix cr because length of 2
Skipping prefix c because length of 1
Skipping prefix cr because length of 2
Skipping prefix c because length of 1
Skipping prefix cr because length of 2
Skipping prefix c because length of 1
Skipping prefix cr because length of 2
Skipping prefix c because length of 1
Skipping prefix cr because length of 2
Skipping prefix c because length of 1
Skipping prefix cr because length of 2
Skipping prefix c because length of 1
Skipping prefix cr because length of 2

We can also run checks on policies to see if they allow dangerous public access

In [9]:
from policyuniverse.policy import Policy
from policyuniverse.statement import ConditionTuple, PrincipalTuple

# An internet accessible policy:
policy01 = dict(
    Version='2012-10-08',
    Statement=dict(
        Effect='Allow',
        Principal='*',
        Action=['rds:*'],
        Resource='*',
        Condition={
            'IpAddress': {
                'AWS:SourceIP': ['0.0.0.0/0']
            }
        }))

policy = Policy(policy01)
assert policy.is_internet_accessible() == False
assert policy.internet_accessible_actions() == set(['rds:*'])

AssertionError: 